In [22]:
from pymongo import MongoClient
import pandas as pd

In [ ]:
client = MongoClient("localhost:27017")
db = client.get_database("ironhack")
db

In [2]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack')

In [4]:
companies = db.get_collection("companies")

In [11]:
companies.find_one()

{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
 'name': 'Wetpaint',
 'permalink': 'abc2',
 'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
 'homepage_url': 'http://wetpaint-inc.com',
 'blog_url': 'http://digitalquarters.net/',
 'blog_feed_url': 'http://digitalquarters.net/feed/',
 'twitter_username': 'BachelrWetpaint',
 'category_code': 'web',
 'number_of_employees': 47,
 'founded_year': 2005,
 'founded_month': 10,
 'founded_day': 17,
 'deadpooled_year': 1,
 'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
 'alias_list': '',
 'email_address': 'info@wetpaint.com',
 'phone_number': '206.859.6300',
 'description': 'Technology Platform Company',
 'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
 'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
 'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and monetize audiences for di

In [23]:
proyec = {"name":1,"category_code": 1, "_id": 0, "offices.city":1}
cond = {"offices.city": "Madrid"}
pd.DataFrame(list(companies.find(cond,proyec)))

,name,category_code,offices
0,Google,search,"[{'city': 'Mountain View'}, {'city': 'Ann Arbo..."
1,Truphone,mobile,"[{'city': 'London'}, {'city': 'New York'}, {'c..."
2,Fon,mobile,[{'city': 'Madrid'}]
3,Buongiorno,mobile,"[{'city': 'Milan'}, {'city': 'Parma'}, {'city'..."
4,Swotti,public_relations,[{'city': 'Madrid'}]
...,...,...,...
78,Genasys,software,[{'city': 'Madrid'}]
79,FRV,cleantech,"[{'city': 'Madrid'}, {'city': 'Madrid'}, {'cit..."
80,StreamUK,public_relations,"[{'city': 'London'}, {'city': 'Madrid'}]"
81,AdLINK Group,advertising,"[{'city': 'Montabaur'}, {'city': 'Munich'}, {'..."
